In [ ]:
# Function to safely install packages
def install_package(package, version=None):
    """Install a package safely, checking if it's already installed."""
    try:
        print(f"Installing {package}")
        if version:
            # Use >= for minimum version specification
            !pip install -q {package}{version}
        else:
            !pip install -q {package}
        print(f"Successfully installed {package}")
        return True
    except Exception as e:
        print(f"Failed to install {package}: {e}")
        return False


In [ ]:

# List of required packages with versions
required_packages = {
    "gradio": "latest",
    "ollama": "latest",
    "markdown2": "latest",
    # "torch": ">=2.4.0",
    # "transformers": ">=4.53.0"
    }

# Install packages
for package, version in required_packages.items():
    if version == "latest":
        install_package(package)
    else:
        install_package(package, version)

print("!!! Installation concluded !!!")


Installing gradio
Installing ollama
Installing markdown2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.2 MB/s eta 0:00:00
!!! Installation concluded !!!


In [ ]:
%%time
import os
import sys
import psutil
import subprocess
import logging
import warnings
import gradio as gr
import ollama
from ollama import chat
from PIL import Image
import io
import base64
import markdown2

CPU times: user 6.21 s, sys: 351 ms, total: 6.56 s
Wall time: 10.5 s


In [ ]:
%%time
# Cell: Logger Configuration
# Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler and set level to debug
file_handler = logging.FileHandler('app.log')
file_handler.setLevel(logging.INFO)

# Create console handler and set level to error
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.ERROR)

# Create formatters
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Add formatters to handlers
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

CPU times: user 215 µs, sys: 948 µs, total: 1.16 ms
Wall time: 4.61 ms


In [ ]:
# ==================================================
# _/$\_\%/_/&\_\@/_/$\_\%/_/&\_\@/_/$\_\%/_/&\_\@/_
# ==================================================
#  **********    System Information   *************
# ==================================================
# _/$\_\%/_/&\_\@/_/$\_\%/_/&\_\@/_/$\_\%/_/&\_\@/_
# ==================================================
def print_system_info(use_torch=False):
    print("System Information:")
    print(f"• Python version: {sys.version}")
    print(f"• Current working directory: {os.getcwd()}")

    if use_torch:
        # No need to import torch. Just use it.
        print(f"• PyTorch version: {torch.__version__}")
        # Check GPU availability and details
        if torch.cuda.is_available():
            gpu_info = {
                "CUDA Available": torch.cuda.is_available(),
                "CUDA Device Count": torch.cuda.device_count(),
                "Current CUDA Device": torch.cuda.current_device(),
                "Device Name": torch.cuda.get_device_name(torch.cuda.current_device()),
                "Memory Allocated (MB)": round(torch.cuda.memory_allocated(0) / 1024**2, 2),
                "Memory Reserved (MB)": round(torch.cuda.memory_reserved(0) / 1024**2, 2),
            }

            print("\n⚡ GPU Detected:")
            for key, value in gpu_info.items():
                print(f"  • {key}: {value}")
        else:
            print("\n😭 No GPU detected. Running on CPU only.")

    # Memory information
    ram = psutil.virtual_memory()
    print("\n🐘 System Memory:")
    print(f"  • Total RAM: {round(ram.total / 1024**2, 2)} MB")
    print(f"  • Available RAM: {round(ram.available / 1024**2, 2)} MB")
    print(f"  • Used RAM: {round(ram.used / 1024**2, 2)} MB")
    print(f"  • RAM Percentage: {ram.percent}% used")

# Check if torch is imported
try:
    import torch
    torch_imported = True  # Indicate that torch is available
except ImportError:
    torch_imported = False  # Indicate that torch is not available

# Call the function based on whether torch is imported
if torch_imported:
    print_system_info(use_torch=True)  # Call the function with use_torch=True
else:
    print_system_info(use_torch=False)  # Call the function with use_torch=False

System Information:
• Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
• Current working directory: /content
• PyTorch version: 2.6.0+cu124

😭 No GPU detected. Running on CPU only.

🐘 System Memory:
  • Total RAM: 12977.95 MB
  • Available RAM: 11611.74 MB
  • Used RAM: 1046.35 MB
  • RAM Percentage: 10.5% used


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

#process = subprocess.Popen("ollama serve", shell=True)

# !ollama pull gemma3n:e4b
# !ollama pull gemma3n:e2b

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
process = subprocess.Popen("ollama serve", shell=True)


In [ ]:
!ollama pull gemma3n:e2b

In [ ]:
data_to_share="""I am a potential customer or a hiring manager. You are Samir LAHOUAR's assistant designed to answer on his behalf **only**  on relevant questions about me and my cv below.
            If a question is not related to his cv or to work jobs, reply politely that you are not allowed to answer. --CV START-- I am Samir LAHOUAR : Solutions Architect | Data Engineer | SAP Specialist | Consultant CDI mon numero de tel : +337 66 41 35 87 , mon email: samirlahouar@gmail.com mon compte upwork : https://www.upwork.com/freelancers/samirlahouar je suis installé à Paris et je suis mobile en france et a l'étranger. je suis libre a partir d'aout 2025 pour travailler. J'ai fait plus de 30 projets sur upwork les plus importants : Data Engineer – Epum
2024 – Maintenant
•	Développement et déploiement de plus de 800 crawlers Python pour extraire des données depuis des sites web complexes, en utilisant des outils avancés comme Playwright, HTTPX et BeautifulSoup.
•	Optimisation des systèmes internes, amélioration des performances et contournement des protections anti-bots, avec des contributions au framework interne pour gérer des structures web sophistiquées.
Solutions Architect – Mediareps LLC
2020 – 2024
•	Mise en place d'un système de protection des droits d'auteur (DMCAForce) : détection des infractions, génération automatique des avis de retrait DMCA, et communication avec les FAI pour garantir la conformité.
•	Conception d'un système RTB (Real-Time Bidding) pour la publicité : gestion des requêtes en temps réel, sélection de la meilleure offre et finalisation des transactions.
Chef de Projet – Auxiliary Teams
2017 – 2020
•	Supervision de projets pour des clients Fortune 500 et startups : déploiement de solutions SaaS intégrant API et interfaces utilisateur.
•	Gestion d'équipes spécialisées en IT, développement web et mobile, avec expertise sur les plateformes cloud Linux et Windows.

Enseignant Chercheur– ENIM
2008 – 2017
•	Enseignement en robotique, apprentissage automatique, vision par ordinateur et impression 3D.
•	Recherche en robotique, planification de trajectoires, apprentissage automatique et vision par ordinateur.
 FORMATIONS
Mastère en Intelligence Artificielle et Digital Management
IA School Paris, 2022 – 2024
•	Gestion de projets IA : méthodologies agiles, optimisation de la prise de décision, conception de solutions data-driven.
•	Analyse de données et interprétation des tendances de marché pour des stratégies marketing efficaces.
Doctorat de robotique
Université de Poitiers, 2004 – 2008
•	Recherche sur la robotique : planification de trajectoires et apprentissage automatique.
•	Maitrise de C/C++, Linux et Temps Réel (RT). OpenCV, ROS, Matlab/Simulink avec utilisation des protocoles de communication CAN, Modbus, RS232/485



COMPÉTENCES TECHNIQUES
Langages de programmation : Python, R, Java, JavaScript, PHP, Bash, Perl
Bases de données : SQL, MySQL, MongoDB, DynamoDB, Elasticsearch
Outils de visualisation : Power BI, Kibana, Grafana, Tableau, Matlab
Cloud Platforms : AWS (EC2, S3, Lambda, CloudFormation, API Gateway), Azure, GCP
DevOps & CI/CD : GitLab CI/CD, Docker Compose, Jenkins, Github Actions
Machine Learning : Scikit-learn, TensorFlow, PyTorch, Keras, Pandas, NumPy, Matplotlib

CERTIFICATIONS
•	Datacamp Data Engineer Associate Certificate
•	MongoDB Data Modeling (M320)
•	MongoDB for Python Developers (M220P)


PROJETS
Système de réponse automatique
•	Développement d’un système basé sur FreeSWITCH et l’API ChatGPT pour répondre aux requêtes clients, augmentant la satisfaction client grâce à des réponses contextuelles et rapides.
Modèle de segmentation d’IRM prostatique
•	Création d’un modèle UNet ajusté pour la segmentation de la prostate, validé sur différentes métriques et résolutions précises.

INFORMATIONS SUPPLÉMENTAIRES
•	Langues : Français (courant), Anglais (professionnel) --CV END--"""

In [ ]:
data_to_share=""
# Unified function for chat with Gemma
def gemma_chat(history, url_input, question):
    try:
        # Validate the question input
        question = question.strip()
        if not question:
            return history, "Please enter a valid question."
        if url_input:  # If a URL has been provided
            message_content = f"You have received a URL for a file: {url_input}. Question: {question}"
        else:  # No URL provided, just process the question
            message_content = f"{data_to_share} Question: {question}"
        # Send the prompt to the Ollama model
        response = chat(model='gemma3n:e2b', messages=[
            {
                "role": "user",
                "content": message_content
            },
        ])
        answer = response['message']['content']
        # Convert the Markdown answer to HTML
        answer_html = markdown2.markdown(answer)
        # Update history with the new interaction
        history.append(f"<div style='color: blue;'>You: {question}</div>")
        history.append(f"<div style='color: green;'>Gemma 3n: {answer_html}</div>")
        history_text = "<br>".join(history)

        return history_text, answer
    except Exception as e:
        return history, f"Error occurred: {str(e)}"


In [ ]:
# Create Gradio interface
with gr.Blocks() as demo:
    history = gr.State([])
    with gr.Column():
        gr.Markdown("# Welcome to Samir LAHOUAR Portal")
        chat_output = gr.HTML(label="Chat History")
        response_output = gr.Textbox(label="Response", placeholder="Model response will appear here...", interactive=False)
        url_input = gr.Textbox(lines=1, label="Enter File URL (audio/image) or leave it blank")
        question_input = gr.Textbox(lines=2, label="Ask Samir's Assistant")

        submit_button = gr.Button("Submit")

    # Connect inputs and outputs
    submit_button.click(
        gemma_chat,
        inputs=[history, url_input, question_input],
        outputs=[chat_output, response_output]
    )

In [ ]:

# Launch the Gradio interface
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4845863e6d8759503c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
